# Deterministic model of three genes interaction network

In this example we are going to perform parameter inference for a deterministic model of three gene interaction, described in (Karlebach & Shamir, 2008). The model is illustrated by the figure below

<img src="threegeneinteractions.png" width="500" height="1000">

Apart from participating in the interactions shown on this diagram, all genes are also activated by the environment and degrade into environment (not shown). Gene 1 is repressed by Gene 3, Gene 2 is activated by Gene 1, and Gene 3 is activated by both Gene 1 and Gene 2.

Using Michaelis-Menten kinetics, we can produce the following system of diferential equations for this system:

$$
\begin{align*}
\frac{dx_1}{dt} &= k_{1,a}\frac{1}{1+k_{3,1} x_3} - k_{1,d} x_1\\
\frac{dx_2}{dt} &= k_{2,a}\frac{k_{1,2} x_1}{1+k_{1,2} x_1} - k_{2,d} x_2\\
\frac{dx_3}{dt} &= k_{3,a}\frac{k_{1,3} x_1 k_{2,3} x_3}{(1+k_{1,3} x_1)(1+k_{2,3} x_2)} - k_{3,d} x_3
\end{align*}
$$

Here, for $i, j \in \{1, 2, 3\}$, $x_i$ denote concentrations of respective genes, as they change with time; $k_{i, a}$ are the rates with which Gene $i$ is activated by the environment; $k_{i, d}$ are the rates with which Gene $i$ degrades into the environment, and $k_{i, j}$ is the rate with which Gene $i$ activates or represses Gene $j$.

### References
- Toni, T., Welch, D., Strelkowa, N., Ipsen, A., & Stumpf, M. P. H. (2009). Approximate Bayesian computation scheme for parameter inference and model selection in dynamical systems. *Interface*, (July 2008), 187–202. https://doi.org/10.1098/rsif.2008.0172
- Karlebach, G., & Shamir, R. (2008). Modelling and analysis of gene regulatory networks. *Nature Reviews Molecular Cell Biology*, 9(10), 770–780. https://doi.org/10.1038/nrm2503


# To run this notebook locally:
* Make sure that [Jupyter](https://jupyter.org/) is installed, along with [Julia](https://www.julialang.org/) and its [Jupyter Kernel](https://github.com/JuliaLang/IJulia.jl).
* Clone or download [GpABC.jl](https://github.com/tanhevg/GpABC.jl) to your machine.
* Run `jupyter` from the `examples` directory of the local copy of `GpABC.jl`:
```bash
$ cd GpABC.jl/examples
$ jupyter notebook abc-examples.ipynb
```
* The first line of the cell below activates the local project environment and downloads all the dependencies.

This notebook was tested under Julia 1.7.2

Mark: to do this, gotta be in normal terminal (not julia, if in julia use exit() to get out). and the directory should be:
```bash
$ cd Documents/Git/GpABC.jl/examples
```

And then now you can run:
```bash
$ jupyter notebook abc-examples.ipynb
```

Then you'll hit run, and it asks for an environment to do this in, I think I just selected Jupyter release channel *this didnt work. I tried again using the jupyter kernel and then selected julia (this is done in the searchbar looking thing above). This seems to be an issue surrounding the working environment that I'm in, perhaps? 

In [ ]:
ENV["PYTHON"]=""; import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate()
using GpABC, OrdinaryDiffEq, Distances, Distributions, Plots, StatsBase, Printf
pyplot()

: 

# Rejection ABC using `GpABC` - estimating parameters

Let's start by implementing the above system of ODEs (model of three genes interaction network) in Julia, and setting up some scaffolding for the ABC computations. The ABC computations will need the user to specify some priors for the parameters they wish to estimate.

The user also needs to specify a function that simulates the model (named `simulator_function` in the cell below) that needs to take the parameters the user wishes to estimate, the variable/non-fixed parameters, and return the ODE solution.

In [ ]:
#
# ABC settings
#
true_params =  [2.0, 1.0, 15.0, 1.0, 1.0, 1.0, 100.0, 1.0, 1.0, 1.0] # nominal parameter values
priors = [Uniform(0.2, 5.), Uniform(0.2, 5.), Uniform(10., 20.),
          Uniform(0.2, 2.), Uniform(0.2, 2.), Uniform(0.2, 2.),
          Uniform(75., 125.), Uniform(0.2, 2.), Uniform(0.2, 2.), 
          Uniform(0.2, 2.)]
param_indices = [1, 3, 9]  #indices of the parameters we want to estimate
priors = priors[param_indices]

#
# ODE solver settings
#
Tspan = (0.0, 10.0)
x0 = [3.0, 2.0, 1.0]
solver = RK4()
saveat = 0.1

#
# Returns the solution to the toy model as solved by OrdinaryDiffEq
#
GeneReg = function(params::AbstractArray{Float64,1},
    Tspan::Tuple{Float64,Float64}, x0::AbstractArray{Float64,1},
    solver::OrdinaryDiffEq.OrdinaryDiffEqAlgorithm, saveat::Float64)

  if size(params,1) != 10
    throw(ArgumentError("GeneReg needs 10 parameters, $(size(params,1)) were provided"))
  end

  function ODE_3GeneReg(dx, x, par, t)
    dx[1] = par[1]/(1+par[7]*x[3]) - par[4]*x[1]
    dx[2] = par[2]*par[8]*x[1]/(1+par[8]*x[1]) - par[5]*x[2]
    dx[3] = par[3]*par[9]*x[1]*par[10]*x[2]./(1+par[9]*x[1])./(1+par[10]*x[2]) - par[6]*x[3]
  end

  prob = ODEProblem(ODE_3GeneReg, x0 ,Tspan, params)
  Obs = solve(prob, solver, saveat=saveat)

  return hcat(Obs.u...)
end

#
# A function that simulates the model 
#
function simulator_function(var_params)
    params = copy(true_params)
    params[param_indices] .= var_params
    GeneReg(params, Tspan, x0, solver, saveat)
end

UndefVarError: UndefVarError: `Uniform` not defined

## Plot reference data

The final scaffolding for the ABC computation that is left to be set up is establishing the reference data, $\mathcal{D}$, which is the gene levels over time. In this example the reference data, $\mathcal{D}$, will be "fake" data for the purpose of setting up an example used for parameter estimation where the true parameter values are known. This is computed by solving the model (`GeneReg`) with the true parameters defined in the above cell.

During parameter estimation using real-life or experimental data, the reference data, $\mathcal{D}$, would be need to be changed from simulated data to the experimental data. This is done through redefining your reference data.

In [ ]:
#
# Get reference data and plot it
#
reference_data = simulator_function(true_params[param_indices])
plot(reference_data', xlabel="t", ylabel="C(t)", linewidth=2, labels=["u1(t)" "u2(t)" "u3(t)"])

## Simulation-based rejection ABC

The intuition beind [rejection ABC](https://tanhevg.github.io/GpABC.jl/latest/overview-abc/#Simulation-based-Rejection-ABC-1) is the following: simulate the ODE for different parameter values sampled from your priors and only accept the parameter combinations that produce ODE simulations *close enough* to your reference data, $\mathcal{D}$. The accepted combinations of parameters serve as an approximation to your posterior of your parameters.

`GpABC.jl` provides two algorithms for rejection ABC: simulation-based and emulation-based. 

Here we begin by demonstrating simulated rejection ABC, by calling [`SimulatedABCRejection`](https://tanhevg.github.io/GpABC.jl/latest/ref-abc/#GpABC.SimulatedABCSMC) function. The simulator function will be called on every step of the  ABC procedure. 

The plot recipe makes it easy to visualise the results. As one can see, after just one iteration of rejection ABC the posterior population does not highlight the true parameters very well. Still, the simualtion ABC results will always serve as the "gold-standard" for the emulation ABC results.

In [ ]:
#
# Simulation
#
n_particles = 2000
threshold = 1.0
sim_result = SimulatedABCRejection(reference_data, simulator_function, priors, threshold, n_particles;
    max_iter=convert(Int, 2e6),
    write_progress=false)
plot(sim_result)

One of the more important choices to be made in the ABC algorithim is the maximum number of iterations. This is the `max_iter` argument to `SimulatedABCRejection` and controls the maximum number of parameters for which the model will be simulated.

If this number is too low then the posteriors will contain too few particles. In fact, the algorithm may terminate before `n_particles` are accepted : `GpABC` will print a warning in this case.

A very high value for `max_iter` is only a problem if a very small number of particles are being accepted (which can happen if `threshold` is too small, for example). In this scenario the ABC algorithm will run for an extremely long time (the exact time depends on the computational cost of the simulator).

The case when `max_iter` is too small is shown below (3,000 vs 2,000,000 in the plot above). These posteriors suggest there is low probability in regions that would be filled with packages where `max_iter` a larger value.

In [ ]:
#
# Simulation with max_iter set too low
#...
n_particles = 2000
threshold = 1.0
sim_result = SimulatedABCRejection(reference_data, simulator_function, priors, threshold, n_particles;
                                   max_iter=convert(Int, 3e3),
                                   write_progress=false)
plot(sim_result)

## Emulation-based rejection ABC

This algorithm is similar to simulation-based procedure described above. Here, instead of running the simulation at each step of the ABC, the simulated results are used to train the emulator that outputs the distance from the given point in the parameter space to reference data. These emulated distances are then used in the ABC algorithm.

Arguments of [`EmulatedABCRejection`](https://tanhevg.github.io/GpABC.jl/latest/ref-abc/#GpABC.EmulatedABCRejection) function are very similar to those of `SimulatedABCRejection`, with the addition of `n_design_points` that controls the number of particles that will be used for simulation and training the emulator.

Note that `EmulatedABCRejection` also has an argument named `max_iter`, but now this refers to the maximum number of emulations. Since the emulator predicts distances in batches with size `batch_size` a single run of an emulated ABC algorithm tests `max_iter`*`batch_size` particles.

In [ ]:
n_design_points = 500
emu_result = EmulatedABCRejection(reference_data, simulator_function, priors, threshold, n_particles, n_design_points;
                                  max_iter=convert(Int, 1e3),
                                  write_progress=false)
plot(emu_result)

# Comparing the resulting posteriors from simulation and emulation

When looking at the approximate posteriors that are from simulation and emulation we are looking to check that they are similar, since emulation's goal is to match simuation.

We can compare these posteriors more quantitatively using the Kullback–Leibler divergence (KLD), which which measures the difference between probability distributions. We estimate the KL from the parameter posterior samples by placing the data into bins and using the discrete estimator. Note this KLD estimate devepend depends on the number of bins and that we are estimating the KLD from the simulation posterior to the emulation one (the KLD is not symmetric).

As one can see the resulting posteriors from simulation and emulation are similar and both center on the correct parameter values.

In [ ]:
nbins = 5 # The number of bins when plotting the posteriors and calcuating KL(simulation || emulation)

function get_bin_edges(samples1, samples2, nbins)
    alldata = vcat([samples1, samples2]...)
    max_val = maximum(alldata)
    min_val = minimum(alldata)
    eps = 1e-9 # So that edge cases are still binned
    Array(range(min_val-eps, stop=max_val+eps, length=nbins+1))
end

function binned_kl_estimate(samples1, samples2, nbins)
    
    function bindata(data)
        bincounts = fit(Histogram, data, get_bin_edges(samples1, samples2, nbins)).weights
        bincounts/sum(bincounts)
    end
    
    eps = 1e-9 # To avoid division by zero
    binned_samples1 = bindata(samples1)
    binned_samples2 = bindata(samples2)
    sum(binned_samples1 .* log.((binned_samples1.+eps)./(binned_samples2.+eps)))
end

kl_divergences = [binned_kl_estimate(sim_result.population[:,j], emu_result.population[:,j], nbins) for j in 1:3]

plts = Array{Plots.Plot}(undef, length(priors))
for idx in 1:length(plts)
    min_pop_len = min(size(emu_result.population, 1), size(sim_result.population, 1))
    plts[idx] = histogram([emu_result.population[1:min_pop_len,idx] sim_result.population[1:min_pop_len,idx]],
        bins=get_bin_edges(emu_result.population[1:min_pop_len,idx], sim_result.population[1:min_pop_len,idx], nbins),
        label=["emulation" "simulation"], fillalpha=0.5, xlabel="Parameter $idx", ylabel="Density",
        title=@sprintf("KL(sim || emu) = %.3f", kl_divergences[idx]))
end
plot(plts..., size=(900,300))

## Particle Acceptance Criteria
 
`GpABC` offers three criteria according to which a particle can be accepted in emulation-based ABC. Given a trained Gaussian process (GP) emulator, these criteria are:

* `MeanEmulatedParticleSelection`: accept the particle if the mean of the GP evaluated at the particle is less than the threshold.
* `MeanVarEmulatedParticleSelection`: accept the particle if the mean of the GP evaluated at the particle is less than the threshold __and__ the corresponding posterior variance is less than the threshold multiplied by a scaling factor. The scaling factor must be specified by the user - the default value is 1.0.
* `PosteriorSampledEmulatedParticleSelection`: draw a sample from the GP posterior evaluated at the particle. Accept the particle if this sample is less than the threshold. If desired, the GP posterior covariance can be approximated by its diagonal elements - set the `use_diagonal_covariance` field of the `PosteriorSampledEmulatedParticleSelection` struct to `true` to use the approximation. The default is to use the full covariance matrix.

Using `MeanEmulatedParticleSelection` is the simplest option but does not consider the variance of the emulator. This could be problematic if you sample particles in regions where your emulator has high variance. The variance may be high because the emulator has not been trained on particles in that region, in which case accepting those particles could lead to inaccurate posteriors.

Select the criterion using the `emulated_particle_selection` argument to `EmulatedABCRejection` (or `EmulatedABCSMC`). The default is `MeanEmulatedParticleSelection`.

Posteriors from emulated ABC Rejection using the three different acceptance criteria are shown below.

In [ ]:
emulation_results = [
    EmulatedABCRejection(reference_data, simulator_function, priors, threshold, n_particles, n_design_points;
            batch_size=100,
            max_iter=convert(Int, 1e3),
            write_progress=false,
            emulated_particle_selection=particle_selection)
    for particle_selection in [MeanEmulatedParticleSelection(), MeanVarEmulatedParticleSelection(), PosteriorSampledEmulatedParticleSelection()]
];

plot([plot(result) for result in emulation_results]..., size=(1500,500), layout=@layout grid(1,3))

Here, the left column of plots uses `MeanEmulatedParticleSelection`, the middle uses `MeanVarEmulatedParticleSelection` (with the defaut scaling factor of 1) and the right-hand plots use `PosteriorSampledEmulatedParticleSelection` (with a full GP posterior covariance). All three criteria lead to similar parameter posteriors in this case.

# ABC-SMC

[ABC-SMC](https://tanhevg.github.io/GpABC.jl/latest/overview-abc/#Simulation-based-ABC-SMC-1) is essentially a sequential set of ABC rejection algorithms, where the population at the previous iteration is used to propose new particles for the current set. The algorithm hones in on the true posterior by incrementally decreasing the threshold at each iteration. The result is that the final population is produced using a much smaller threshold than is possible in ABC Rejection, since the particles proposed according to a previous ABC iteration are more likely to match the data well than those sampled from the priors.

In [ ]:
#
# ABC-SMC settings
#
threshold_schedule = [3.0, 2.0, 1.0, 0.5, 0.2];

## Simulation-based ABC-SMC

Similarly to rejection ABC, two algorithms are provided for ABC-SMC: [simulation](https://tanhevg.github.io/GpABC.jl/latest/overview-abc/#Simulation-based-ABC-SMC-1) and [emulation](https://tanhevg.github.io/GpABC.jl/latest/overview-abc/#Emulation-based-ABC-SMC-1) based. Parameters for [`SimulatedABCSMC`](https://tanhevg.github.io/GpABC.jl/latest/ref-abc/#GpABC.SimulatedABCSMC) function are identical to those of `SimulatedABCRejection`. Lower thresholds allow the SMC algorithm to produce much more accurate results than rejection ABC - note the ranges on the X-axes of the histograms below. This, however, is achieved at the expense of a longer run time.

In [ ]:
population_colors=["#FF2F4E", "#D0001F", "#A20018", "#990017", "#800013"]
sim_abcsmc_res = SimulatedABCSMC(reference_data,
    simulator_function,
    priors,
    threshold_schedule,
    n_particles; 
    write_progress=false)
plot(sim_abcsmc_res, population_colors=population_colors)

## Emulation-based ABC-SMC

For [`EmulatedABCSMC`](https://tanhevg.github.io/GpABC.jl/latest/ref-abc/#GpABC.EmulatedABCSMC) we have the additional option to retrain the emulator between each population using particles sampled from the most recent population.

In [ ]:
emu_abcsmc_res = EmulatedABCSMC(reference_data,
    simulator_function,
    priors,
    threshold_schedule,
    n_particles,
    n_design_points;
    batch_size=500,
    write_progress=false,
    emulator_retraining = PreviousPopulationThresholdRetraining(n_design_points, 100, 10),
    emulated_particle_selection = MeanVarEmulatedParticleSelection()
    )

plot(emu_abcsmc_res, population_colors=population_colors)

## Summary plot of ABC-SMC emulation and simulation results.

As in the paper. Subplots on the diagonal and below show marginal and joint posterior distributions of parameter estimates in the final ABC-SMC population (simulation in blue and emulation in red). Scatterplots above the diagonal show intermediate ABC-SMC populations with GP emulations; darker colour indicates decreasing threshold Black dashed lines indicate the true parameter values.

In [ ]:
plot(emu_abcsmc_res, sim_abcsmc_res, true_params[param_indices])

## Compare the posteriors

Here we compare the parameter posteriors found by ABC-SMC at the final population. Again we use a binned estimate of the KL divergence to quantify the distance from the simulation posterior to the emulation one.

In [ ]:
nbins = 5
kl_divergences = [binned_kl_estimate(sim_abcsmc_res.population[end][:,j], emu_abcsmc_res.population[end][:,j], nbins) for j in 1:3]

sim_pop = sim_abcsmc_res.population[end]
emu_pop = emu_abcsmc_res.population[end]
for idx in 1:length(plts)
    min_pop_len = min(size(emu_pop, 1), size(sim_pop, 1))
    plts[idx] = histogram([emu_pop[1:min_pop_len,idx] sim_pop[1:min_pop_len,idx]],
        bins=get_bin_edges(emu_pop[1:min_pop_len,idx], sim_pop[1:min_pop_len,idx], nbins),
        label=["emulation" "simulation"], fillalpha=0.5, xlabel="Parameter $idx", ylabel="Density",
        title=@sprintf("KL(sim || emu) = %.4f", kl_divergences[idx]))
end

plot(plts..., size=(900,300))

Again, the posteriors from emulation both center on the correct parameter values. According to the KLD estimates the first parameter posteriors from simulation and emulation are similar, but the KLD values for the other two parameters indicate that they are not as similar. This is also apparent from the histograms.

# Custom emulator training procedure

The snippet below demonstrates subtyping of `AbstractEmulatorTraining` and usage of `gp_train` for custom emulator training. As mentioned in [`gp_train` reference](https://tanhevg.github.io/GpABC.jl/latest/ref-gp/), all hyperparameters are bounded between $e^{-10}$ and $e^{10}$ during GP training. Suppose this is not good enough for us, and we want control the noise variance $\sigma_n$ more tightly, and let it vary only from 0.001 to 1. In total we need to optimise 5 hyperparameters: signal variance, noise variance, and three length scales (by the number of ODE parameters that we are estimating). $\sigma_n$ is always the last hyperparameter, and the custom optimisation constraints and starting point for it can be specified as follows:

In [ ]:
struct CustomEmulatorTraining <: AbstractEmulatorTraining end
import GpABC.train_emulator
function train_emulator(training_x::AbstractArray{T, 2}, y::AbstractArray{T, 2}, emulator_training::CustomEmulatorTraining) where {T<:Real}
    @info "Custom Emulator Training"
    # make sure the starting point is within bounds
    optim_start = vcat(ones(4), 0.5)
    lower_bound = vcat(exp(-10) .* ones(4), 0.001)
    upper_bound = vcat(exp(10) .* ones(4), 1.0)
    gpem = GPModel(training_x=training_x, training_y=y, 
        kernel=SquaredExponentialArdKernel(), 
        gp_hyperparameters=optim_start) 
    gp_train(gpem, hp_lower=lower_bound, hp_upper=upper_bound)
    gpem
end
emu_abcsmc_res_custom = EmulatedABCSMC(reference_data,
    simulator_function,
    priors,
    threshold_schedule,
    n_particles,
    n_design_points;
    batch_size=500,
    write_progress=false,
    emulator_retraining = PreviousPopulationThresholdRetraining(n_design_points, 100, 10),
    emulated_particle_selection = MeanVarEmulatedParticleSelection(),
    # this instructs the ABC-SMC procedure to use the custom training implemented above
    emulator_training=CustomEmulatorTraining() 
    )
plot(emu_abcsmc_res_custom, sim_abcsmc_res, true_params[param_indices])